In [1]:
import cv2,os
import numpy as np
from matplotlib import pyplot as plt
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from tqdm import tqdm

from torch.nn import functional as F
from collections import OrderedDict
import math

Max_shape_0=256
Max_shape_1=256
device = "cuda"

# load model and data

In [2]:
# load model
PATH = "./qsub220208/Model_Resnet18_nomask.pkl"
model = torch.load(PATH)

In [3]:
# load data
imgdir0120 = "../../Datasets/211202NDAcquisition/CellsNoMask/NDAcquisition-01/"
imgdir0220 = "../../Datasets/211202NDAcquisition/CellsNoMask/NDAcquisition-02Nami_x20/"
imgdir0140 = "../../Datasets/211202NDAcquisition/CellsNoMask/NDAcquisition-01x40/"
imgdir0240 = "../../Datasets/211202NDAcquisition/CellsNoMask/NDAcquisition-02Nami_x40/"

imgnamelist = ["NDAcquisition-01_XY001_1","NDAcquisition-01_XY001_8",
           "NDAcquisition-01x40_XY0001_1","NDAcquisition-01x40_XY0001_11",
           "NDAcquisition-02Nami_x20_XY009_1","NDAcquisition-02Nami_x20_XY136_2",
           "NDAcquisition-02Nami_x40_XY001_2","NDAcquisition-02Nami_x40_XY127_2"]

imgdirlist = [imgdir0120,imgdir0120,imgdir0140,imgdir0140,imgdir0220,imgdir0220,imgdir0240,imgdir0240]
label=[0,0,0,0,1,1,1,1]

In [5]:
# 定义钩子函数，获取指定层名称的特征
feature_activation = {} # 保存获取的输出
def get_activation(name):
    def hook(model, input, output):
        feature_activation[name] = output.detach()
    return hook


def pad(img_path):
    img = cv2.imread(img_path)
    imgSize = img.shape
    top_size,bottom_size = (Max_shape_0-imgSize[0])//2,(Max_shape_0-imgSize[0])//2
    left_size,right_size = (Max_shape_1-imgSize[1])//2,(Max_shape_1-imgSize[1])//2
    if (imgSize[0] % 2) != 0:
        top_size,bottom_size = (Max_shape_0-imgSize[0])//2,(Max_shape_0-imgSize[0])//2+1
    if (imgSize[1] % 2) != 0:     
        left_size,right_size = (Max_shape_1-imgSize[1])//2,(Max_shape_1-imgSize[1])//2+1
    imgpad = cv2.copyMakeBorder(img,top_size,bottom_size,left_size,right_size,cv2.BORDER_CONSTANT,value=(0,0,0))
    return imgpad


def dataTransform(img_path):
    imgpad = pad(img_path)
    transform = transforms.Compose([transforms.ToTensor()])
    input_img = transform(imgpad).unsqueeze(0).to(device)
    return input_img


def getfeature(img_path,plt):
    img = cv2.imread(img_path)
    imgSize = img.shape
    top_size,bottom_size = (Max_shape_0-imgSize[0])//2,(Max_shape_0-imgSize[0])//2
    left_size,right_size = (Max_shape_1-imgSize[1])//2,(Max_shape_1-imgSize[1])//2
    if (imgSize[0] % 2) != 0:
        top_size,bottom_size = (Max_shape_0-imgSize[0])//2,(Max_shape_0-imgSize[0])//2+1
    if (imgSize[1] % 2) != 0:     
        left_size,right_size = (Max_shape_1-imgSize[1])//2,(Max_shape_1-imgSize[1])//2+1
    imgpad = cv2.copyMakeBorder(img,top_size,bottom_size,left_size,right_size,cv2.BORDER_CONSTANT,value=(0,0,0))
    transform = transforms.Compose([transforms.ToTensor()])
    input_img = transform(imgpad).unsqueeze(0).to(device)
    for name, layer in model.named_modules():
        layer.register_forward_hook(get_activation(name))
    output = model(input_img)
    feature_activation.pop("module.avgpool")
    feature_activation.pop("module.fc.0")
    feature_activation.pop("module.fc.1")
    feature_activation.pop("module.fc.2")
    feature_activation.pop("module.fc.3")
    feature_activation.pop("module.fc.4")
    feature_activation.pop("module.fc")
    feature_activation.pop("module")
    feature_activation.pop("")
    
    if plt == True:
        for key in feature_activation:
            bn = feature_activation[key].cpu()
            print(key," : ",bn.shape)
            s = int(imgpad.shape[0]/bn.shape[2])
            n = math.ceil(math.sqrt(bn.shape[1]))
            plt.figure(figsize=(20,20))
            for i in range(bn.shape[1]):
                plt.subplot(n,n,i+1)
                plt.imshow(bn[0,i,
                                  int(top_size/s):int((top_size+imgSize[0])/s),
                                  int(left_size/s):int((left_size+imgSize[1])/s)], cmap='gray')
                plt.axis('off')
            plt.show()

In [6]:
# check predict label correct or not
for i in range(8):
    imgdir = imgdirlist[i]
    imgname = imgnamelist[i]
    img_path = imgdir + imgname + ".tif"
    imgpad = pad(img_path)
    transform = transforms.Compose([transforms.ToTensor()])
    input_img = transform(imgpad).unsqueeze(0).to(device)
    
    output = model(input_img)
    y=output.argmax(1).cpu()
    print(y==label[i],"  Real: ",label[i],";   Predict: ", y)

tensor([True])   Real:  0 ;   Predict:  tensor([0])
tensor([True])   Real:  0 ;   Predict:  tensor([0])
tensor([True])   Real:  0 ;   Predict:  tensor([0])
tensor([False])   Real:  0 ;   Predict:  tensor([1])
tensor([True])   Real:  1 ;   Predict:  tensor([1])
tensor([True])   Real:  1 ;   Predict:  tensor([1])
tensor([True])   Real:  1 ;   Predict:  tensor([1])
tensor([True])   Real:  1 ;   Predict:  tensor([1])


In [7]:
i=0
imgdir = imgdirlist[i]
imgname = imgnamelist[i]
img_path = imgdir + imgname + ".tif"
getfeature(img_path,False)

In [8]:
for key in feature_activation:
    bn = feature_activation[key].cpu()
    print(key," : ",bn.shape)

module.conv1  :  torch.Size([1, 64, 128, 128])
module.bn1  :  torch.Size([1, 64, 128, 128])
module.relu  :  torch.Size([1, 64, 128, 128])
module.maxpool  :  torch.Size([1, 64, 64, 64])
module.layer1.0.conv1  :  torch.Size([1, 64, 64, 64])
module.layer1.0.bn1  :  torch.Size([1, 64, 64, 64])
module.layer1.0.relu  :  torch.Size([1, 64, 64, 64])
module.layer1.0.conv2  :  torch.Size([1, 64, 64, 64])
module.layer1.0.bn2  :  torch.Size([1, 64, 64, 64])
module.layer1.0  :  torch.Size([1, 64, 64, 64])
module.layer1.1.conv1  :  torch.Size([1, 64, 64, 64])
module.layer1.1.bn1  :  torch.Size([1, 64, 64, 64])
module.layer1.1.relu  :  torch.Size([1, 64, 64, 64])
module.layer1.1.conv2  :  torch.Size([1, 64, 64, 64])
module.layer1.1.bn2  :  torch.Size([1, 64, 64, 64])
module.layer1.1  :  torch.Size([1, 64, 64, 64])
module.layer1  :  torch.Size([1, 64, 64, 64])
module.layer2.0.conv1  :  torch.Size([1, 128, 32, 32])
module.layer2.0.bn1  :  torch.Size([1, 128, 32, 32])
module.layer2.0.relu  :  torch.Size

In [ ]:
for i in range(4,8):
    imgdir = imgdirlist[i]
    imgname = imgnamelist[i]
    img_path = imgdir + imgname + ".tif"
    getfeature(img_path,False)
    imgpad,input_img,imgSize,top_size,bottom_size,left_size,right_size = dataTransform(img_path)
    
     
    for key in feature_activation:
        bn = feature_activation[key].cpu()
        bn = np.array(bn)
        s = int(imgpad.shape[0]/bn.shape[2])
        n = math.ceil(math.sqrt(bn.shape[1]))
        plt.figure(figsize=(20,20))
        
        # save path
        path = "./Feature18/" + imgname + "/" + str(key)
        if not os.path.exists(path):
            os.makedirs(path)
            
        for i in range(bn.shape[1]):
            # save each feature in each layers
            savename = path+"/"+str(key)+"_"+str(i)+".png"
            plt.imsave(savename, bn[0,i,int(top_size/s):int((top_size+imgSize[0])/s),
                                  int(left_size/s):int((left_size+imgSize[1])/s)])
        
            # save all feature in each layers
            plt.subplot(n,n,i+1)
            plt.imshow(bn[0,i,int(top_size/s):int((top_size+imgSize[0])/s),
                              int(left_size/s):int((left_size+imgSize[1])/s)], cmap='gray')
        savename = path+"/"+str(key)+".png"
        plt.savefig(savename)
        print("Successfully saved ", savename)

## Discussion

In [ ]:
%rm ./qsub220209/FeatureView/*

In [ ]:
for key in feature_activation:
    print(">>> ",key)
    n = 1
    plt.figure(figsize=(20,20))
    for i in range(0,8,2):
        img=cv2.imread("./qsub220209/Feature18/"+imgnamelist[i]+"/"+key+".png")
        plt.subplot(2,2,n)
        plt.imshow(img, cmap='gray')
        plt.axis("off")
        n+=1
#     plt.show()
    plt.savefig("./qsub220209/FeatureView/"+key+".png")

# HeatMap可視化

## GradCAM

## Save all feature map 

In [10]:
# for i in range(8):
#     imgdir = imgdirlist[i]
#     imgname = imgnamelist[i]
#     img_path = imgdir + imgname + ".tif"
#     getfeature(img_path,False)
#     imgpad,input_img,imgSize,top_size,bottom_size,left_size,right_size = dataTransform(img_path)
    
     
#     for key in feature_activation:
#         bn = feature_activation[key].cpu()
#         bn = np.array(bn)
#         s = int(imgpad.shape[0]/bn.shape[2])
#         n = math.ceil(math.sqrt(bn.shape[1]))
#         plt.figure(figsize=(20,20))
        
#         # save path
#         path = "./Feature18/" + imgname + "/" + str(key)
#         if not os.path.exists(path):
#             os.makedirs(path)
            
#         for i in range(bn.shape[1]):
#             # save each feature in each layers
#             savename = path+"/"+str(key)+"_"+str(i)+".png"
#             plt.imsave(savename, bn[0,i,int(top_size/s):int((top_size+imgSize[0])/s),
#                                   int(left_size/s):int((left_size+imgSize[1])/s)])
        
#             # save all feature in each layers
#             plt.subplot(n,n,i+1)
#             plt.imshow(bn[0,i,int(top_size/s):int((top_size+imgSize[0])/s),
#                               int(left_size/s):int((left_size+imgSize[1])/s)], cmap='gray')
#         savename = path+"/"+str(key)+".png"
#         plt.savefig(savename)
#         print("Successfully saved ", savename)